# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import csv
csv_path = "Resources/output_data_file"
# read csv
weather_df = pd.read_csv(csv_path)
weather_df


,city,lat,lon,temp,humidity,cloudiness,windspeed,country,date
0,Khuldābād,20.0500,75.1833,55.40,54,16,6.73,IN,1612657074
1,Albany,42.6001,-73.9662,28.00,41,5,4.88,US,1612656902
2,Soe,-9.8607,124.2840,73.38,83,94,6.87,ID,1612657078
3,Atasū,48.6814,71.6433,20.80,95,88,25.14,KZ,1612657079
4,Lebu,-37.6167,-73.6500,59.18,91,64,2.17,CL,1612657080
...,...,...,...,...,...,...,...,...,...
579,Yaring,6.8662,101.3689,73.40,88,20,4.61,TH,1612657798
580,Cham,47.1821,8.4636,42.01,100,69,2.30,CH,1612657799
581,Safi,32.2994,-9.2372,55.81,71,49,7.00,MA,1612657740
582,Burgersdorp,-30.9977,26.3286,63.88,79,1,3.96,ZA,1612657804


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["lat", "lon"]].astype(float)

# Convert humidity to float and store
humidity = weather_df["humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

hotel_df = weather_df.loc[weather_df['temp']> 70].loc[weather_df['temp']<80] \
.loc[weather_df['windspeed']< 10].loc[weather_df['cloudiness']== 0]

hotel_df




,city,lat,lon,temp,humidity,cloudiness,windspeed,country,date
10,Mataura,-46.1927,168.8643,77.25,50,0,2.44,NZ,1612657087
163,Lázaro Cárdenas,17.9583,-102.2000,79.32,64,0,4.94,MX,1612657291
335,Hambantota,6.1241,81.1185,71.60,100,0,3.44,LK,1612657499
364,Maceió,-9.6658,-35.7353,77.00,88,0,9.22,BR,1612657419
409,Mercedes,-34.6515,-59.4307,75.00,70,0,1.99,AR,1612657589
470,Pilar,-9.5972,-35.9567,77.00,88,0,9.22,BR,1612657668
474,Vao,-22.6667,167.4833,77.58,81,0,4.52,NC,1612657672


In [ ]:
# ### Hotel Map
# * Store into variable named `hotel_df`.

# * Add a "Hotel Name" column to the DataFrame.
# * Set parameters to search for hotels with 5000 meters.
# * Hit the Google Places API for each city's coordinates.
# * Store the first Hotel result into the DataFrame.
# * Plot markers on top of the heatmap.

In [7]:
# find the closest hotel for each set of coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat, lon from df
    lat = row["lat"]
    lon = row["lon"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{lon}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 10: Mataura.
Closest hotel is Ellie's Villa.
------------
Retrieving Results for Index 163: Lázaro Cárdenas.
Closest hotel is Collection O Hotel Quinta Antigua.
------------
Retrieving Results for Index 335: Hambantota.
Closest hotel is Bungalow 63.
------------
Retrieving Results for Index 364: Maceió.
Closest hotel is Hotel Ponta Verde Maceió.
------------
Retrieving Results for Index 409: Mercedes.
Closest hotel is Posada del Inti Centro.
------------
Retrieving Results for Index 470: Pilar.
Closest hotel is Pousada e churrascaria Manguaba.
------------
Retrieving Results for Index 474: Vao.
Closest hotel is Hotel Kou-Bugny.
------------


In [8]:
hotel_df

,city,lat,lon,temp,humidity,cloudiness,windspeed,country,date,Hotel Name
10,Mataura,-46.1927,168.8643,77.25,50,0,2.44,NZ,1612657087,Ellie's Villa
163,Lázaro Cárdenas,17.9583,-102.2000,79.32,64,0,4.94,MX,1612657291,Collection O Hotel Quinta Antigua
335,Hambantota,6.1241,81.1185,71.60,100,0,3.44,LK,1612657499,Bungalow 63
364,Maceió,-9.6658,-35.7353,77.00,88,0,9.22,BR,1612657419,Hotel Ponta Verde Maceió
409,Mercedes,-34.6515,-59.4307,75.00,70,0,1.99,AR,1612657589,Posada del Inti Centro
470,Pilar,-9.5972,-35.9567,77.00,88,0,9.22,BR,1612657668,Pousada e churrascaria Manguaba
474,Vao,-22.6667,167.4833,77.58,81,0,4.52,NC,1612657672,Hotel Kou-Bugny


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
plt.savefig('hotel_heatmap.png')

<Figure size 432x288 with 0 Axes>